In [1]:
#Most of the blockchain-creation code was from 
# https://dzone.com/articles/how-to-create-your-own-cryptocurrency-blockchain-i 
#and edited to fit my idea, it is not a real and functional blockchain as there is no proof-of-work algorithm. 
#The code that isn't the Block/BlockChain classes is dominantly my own work. 

In [44]:
#RUN THIS FIRST, ONCE

import hashlib
import time
import pandas as pd

class Block(object):

    def __init__(self, index, proof_number, previous_hash, data, timestamp=None):
        self.index = index
        self.proof_number = proof_number
        self.previous_hash = previous_hash
        self.data = data
        self.timestamp = timestamp or time.time()

    @property
    def compute_hash(self):
        string_block = "{}{}{}{}{}".format(self.index, self.proof_number, self.previous_hash, self.data, self.timestamp)
        return hashlib.sha256(string_block.encode()).hexdigest()

    def __repr__(self):
        return "{} - {} - {} - {} - {}".format(self.index, self.proof_number, self.previous_hash, self.data, self.timestamp)

class BlockChain(object):

    def __init__(self):
        self.chain = []
        self.current_data = []
        self.nodes = set()
        self.build_genesis()

    def build_genesis(self):
        self.build_block(proof_number=0, previous_hash=0)

    def build_block(self, proof_number, previous_hash):
        block = Block(
            index=len(self.chain),
            proof_number=proof_number,
            previous_hash=previous_hash,
            data=self.current_data
        )

        self.current_data = []
        self.chain.append(block)
        return block

    
    @staticmethod
    def confirm_validity(block, previous_block):

        if previous_block.index + 1 != block.index:
            return False

        elif previous_block.compute_hash != block.previous_hash:
            return False

        elif block.timestamp != previous_block.timestamp:
            return False

        return True

    def get_data(self, data, sender, receiver, amount, previous_hash, block):
        self.current_data.append({
            'data': data,
            'sender': sender,
            'receiver': receiver,
            'amount': amount,
            'previous_hash' : previous_hash,
            'block' : block
        })
        return True        
    
    
    @staticmethod
    def proof_of_work(last_proof):
        pass

    @property
    def latest_block(self):
        return self.chain[-1]

    def chain_validity(self):
        pass        

    def block_mining(self, details_miner):       
        self.get_data(
            data={},
            sender="0", #it implies that this node has created a new block
            receiver=details_miner,
            amount=1, #creating a new block (or identifying the proof number) is awared with 1
            previous_hash=self.latest_block.compute_hash,
            block=self.latest_block
        )
        last_block = self.latest_block
        last_proof_number = last_block.proof_number
        proof_number = self.proof_of_work(last_proof_number)
        last_hash = last_block.compute_hash
        block = self.build_block(proof_number, last_hash)
        return vars(block)  

    def create_node(self, address):
        self.nodes.add(address)
        return True

    @staticmethod
    def get_block_object(block_data):        
        return Block(
            block_data['index'],
            block_data['proof_number'],
            block_data['previous_hash'],
            block_data['data'],
            timestamp=block_data['timestamp']
        )



In [45]:
#RUN THIS ONCE

blockchain = BlockChain()

last_block = blockchain.latest_block

last_proof_number = last_block.proof_number

proof_number = blockchain.proof_of_work(last_proof_number)



In [79]:
#RUN THIS MULTIPLE TIMES WITH DIFFERENT FILENAMES TO SIMULATE CHANGING DATA

import pandas as pd

file = input("Filename, case sensitive: ")
students = pd.read_csv (file)
print(students)
students=pd.DataFrame(students)

Filename, case sensitive: class2012.csv
      id            last      first   year   month  degree      major
0  36456   Cucumberpatch   Bunnyboy   2012       7     Bsc     acting
1  98765            Wash      Hoban   2012      12     PhD      pilot
2  56565           Obama     Barack   2012      12     PhD   politics


In [80]:
#RUN THIS MULTIPLE TIMES AFTER RUNNING THE ABOVE TO SIMULATE CHANGING DATA

print("STARTING")
print(blockchain.chain)

blockchain.get_data(
    data = students,
    sender="0", 
    receiver="Blockchain Bootcamp", 
    amount=1,
    previous_hash = last_block.compute_hash,
    block = blockchain.build_block(proof_number, last_proof_number))
    
print("CREATION SUCCESSFUL")
print(blockchain.chain)

STARTING
[0 - 0 - 0 - [] - 1608600481.1907825, 1 - None - 0 - [] - 1608600487.5576644, 2 - None - 0b0384ddea887282a95bca060e9cc4c1110971d1097a7a378b690a13af70677a - [{'data':       id            last      first   year   month  degree      major
0  36456   Cucumberpatch   Bunnyboy   2012       7     Bsc     acting
1  98765            Wash      Hoban   2012      12     PhD      pilot
2  56565           Obama     Barack   2012      12     PhD   politics, 'sender': '0', 'receiver': 'Blockchain Bootcamp', 'amount': 1, 'previous_hash': '58245104c3a6995106a4ecd5422ff4757378e5fee194d2719cfad445de524996', 'block': 1 - None - 0 - [] - 1608600487.5576644}, {'data': {}, 'sender': '0', 'receiver': 'Blockchain Bootcamp', 'amount': 1, 'previous_hash': '0b0384ddea887282a95bca060e9cc4c1110971d1097a7a378b690a13af70677a', 'block': 1 - None - 0 - [] - 1608600487.5576644}] - 1608600489.8307514, 3 - None - 0 - [] - 1608600544.6326654, 4 - None - 0 - [{'data':       id   last    first   year   month  degree 

In [81]:
#This is just a visual to show what's currently in the chain, that the hashes exist and it's working as intended
#This isn't something the general users would get to see

print(blockchain.current_data, "\n\n")
print(len(blockchain.chain)-1)
for x in range(len(blockchain.chain)-1):
    print(x, ":\n", blockchain.chain[x])

[{'data':       id            last      first   year   month  degree      major
0  36456   Cucumberpatch   Bunnyboy   2012       7     Bsc     acting
1  98765            Wash      Hoban   2012      12     PhD      pilot
2  56565           Obama     Barack   2012      12     PhD   politics, 'sender': '0', 'receiver': 'Blockchain Bootcamp', 'amount': 1, 'previous_hash': '58245104c3a6995106a4ecd5422ff4757378e5fee194d2719cfad445de524996', 'block': 7 - None - 0 - [] - 1608601078.050118}] 


7
0 :
 0 - 0 - 0 - [] - 1608600481.1907825
1 :
 1 - None - 0 - [] - 1608600487.5576644
2 :
 2 - None - 0b0384ddea887282a95bca060e9cc4c1110971d1097a7a378b690a13af70677a - [{'data':       id            last      first   year   month  degree      major
0  36456   Cucumberpatch   Bunnyboy   2012       7     Bsc     acting
1  98765            Wash      Hoban   2012      12     PhD      pilot
2  56565           Obama     Barack   2012      12     PhD   politics, 'sender': '0', 'receiver': 'Blockchain Bootcamp'

In [93]:
#This gives you a csv record of all previous graduates


output=pd.DataFrame()
for x in range(len(blockchain.chain)-1):
    if (len(blockchain.chain[x].data) >= 1):
        output = output.append(blockchain.chain[x].data[0].get("data"), ignore_index=True)
output = output.append(blockchain.current_data[0].get("data"), ignore_index=True)

output
output.dropna(how='all', inplace=True)
output["id"] = output.id.astype(int)
with open('Graduates.txt', 'a') as f:
    f.write(
        output.to_string(header = True, index = True)
    )
print("Export successful!\n")

Export successful!



In [86]:
#This is what gives you the data in a useable way

for x in range(len(blockchain.chain)-1):
    #print(blockchain.chain[x].data)
    if (len(blockchain.chain[x].data) >= 1):
        print(blockchain.chain[x].data[0].get("data"))
        
print("\n\nCurrent Data Block:\n", blockchain.current_data[0].get("data"))

      id            last      first   year   month  degree      major
0  36456   Cucumberpatch   Bunnyboy   2012       7     Bsc     acting
1  98765            Wash      Hoban   2012      12     PhD      pilot
2  56565           Obama     Barack   2012      12     PhD   politics
      id   last    first   year   month  degree     major
0  66666   Name   Holder   2019       7     Bsc   Physics
1  98989   Test   Person   2019       6     PhD   Physics
      id      last            first   year   month  degree            major
0  12345      Abel           Tasman   2020      12     Bsc        astronomy
1  54321      Cook          Captain   2020      12     Bsc          weather
2  10001   Hillary       Sir Edmund   2020       7     PhD   mountaineering
3  10000     Putin         Vladimir   2020      12     PhD             math
4  10101    Sherpa   Tenzing Norgay   2020       7     PhD   mountaineering
5  77777   Someone             Here   2020      10     Bsc          cooking
{}


Current D

In [94]:
#This is a rudimentary search function based on the student's unique ID number

id_num = input("Student id number: ")
print(output[output['id'].astype(str).str.contains(id_num)])

Student id number: 66666
      id   last    first    year   month  degree     major
3  66666   Name   Holder  2019.0     7.0     Bsc   Physics


In [77]:
#Theretically works to mine the data, but without the proof-of-work it just kinda... gets the data. easily.
blockchain.block_mining("Blockchain Bootcamp")

{'index': 6,
 'proof_number': None,
 'previous_hash': 'baa4ba4ba226eb710a58fe9aa58b030f0c71fa84564fe3e8fe2b8ceab5b01382',
 'data': [{'data': {},
   'sender': '0',
   'receiver': 'Blockchain Bootcamp',
   'amount': 1,
   'previous_hash': 'baa4ba4ba226eb710a58fe9aa58b030f0c71fa84564fe3e8fe2b8ceab5b01382',
   'block': 5 - None - 608eb0b912905dd32ff55fd4f9838470651a7dc5118c06e50cde53ae727b1ee7 - [{'data':       id      last            first   year   month  degree            major
   0  12345      Abel           Tasman   2020      12     Bsc        astronomy
   1  54321      Cook          Captain   2020      12     Bsc          weather
   2  10001   Hillary       Sir Edmund   2020       7     PhD   mountaineering
   3  10000     Putin         Vladimir   2020      12     PhD             math
   4  10101    Sherpa   Tenzing Norgay   2020       7     PhD   mountaineering
   5  77777   Someone             Here   2020      10     Bsc          cooking, 'sender': '0', 'receiver': 'Blockchain Bootc